# 1. 데이터 로드

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import ttest_rel
from sklearn.tree import plot_tree
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv("train.csv")
x = data.drop(['ID', 'Cancer'], axis=1)
y = data['Cancer']

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.45, random_state=42)

# 2. 전처리기 정의

In [2]:
class add_new_feature(BaseEstimator, TransformerMixin):
    def t4_category(self, x):
        if x < 6:
            return 'T4_Low'
        elif x > 11.98:
            return 'T4_High'
        else:
            return 'T4_Normal'
    def t3_category(self, x):
        if x < 1.4:
            return 'T3_Low'
        elif x > 3:
            return 'T3_High'
        else:
            return 'T3_Normal'
    def tsh_category(self, x):
        if x < 0.27:
            return 'TSH_Low'
        elif x > 4.2:
            return 'TSH_High'
        else:
            return 'TSH_Normal'
    
    def age_category(self, x):
        if x < 30:
            return 'Young'
        elif x < 50:
            return 'Middle'
        elif x < 65:
            return 'Senior'
        else:
            return 'Elderly'
    
    def nodule_size_category(self, x):
        if x < 1.0:
            return 'Small'
        elif x < 2.0:
            return 'Medium'
        elif x < 4.0:
            return 'Large'
        else:
            return 'VeryLarge'
    
    def fit(self, x, y=None):
        return self
    def transform(self, x, y=None):
        x = x.copy()
        
        x['T3_Result_Cat'] = pd.cut(x['T3_Result'], 
                                   bins=[-np.inf, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, np.inf],
                                   labels=[0, 1, 2, 3, 4, 5, 6, 7])
        
        x['T3_Cat'] = x['T3_Result'].apply(lambda x : self.t3_category(x))
        x['T4_Cat'] = x['T4_Result'].apply(lambda x : self.t4_category(x))
        
        x['Race&T3'] = x['Race'].astype(str) + x['T3_Result_Cat'].astype(str)
        x['Family&Iodine'] = x['Family_Background'].astype(str) + x['Iodine_Deficiency'].astype(str)
        x['T3&T4'] = x['T3_Cat'].astype(str) + x['T4_Cat'].astype(str)
        return x

class dropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, x, y=None):
        return self
        
    def transform(self, x, y=None):
        x = x.copy()
        return x.drop(self.columns, axis=1)

class custom_encoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = None
        self.num_feature = None
        self.cat_feature = None
        self.OneHot_feature = None
    def fit(self, x, y=None):
        self.num_feature = x.select_dtypes('number').columns.to_list()
        self.cat_feature = x.select_dtypes(['object', 'category']).columns.to_list()
        self.encoder = ColumnTransformer([
            ('OneHot', OneHotEncoder(sparse_output=False), self.cat_feature),
            ('Scaler', StandardScaler(), self.num_feature)
        ])
        self.encoder.fit(x)
        self.OneHot_feature = self.encoder.named_transformers_['OneHot'].get_feature_names_out(self.cat_feature).tolist()
        
        return self
    def transform(self, x, y=None):
        x = x.copy()
        encoded = self.encoder.transform(x)
        
        return pd.DataFrame(
            encoded,
            columns=self.OneHot_feature + self.num_feature
        )

# 파이프라인 구성 
useless_feature = ['T3_Result', 'T4_Result', 'TSH_Result', 
                   'Age', 'Nodule_Size', 
                   'T3_Cat', 'T4_Cat', 'T3_Result_Cat']
preprocessor = Pipeline([
    ('add_new_feature', add_new_feature()),
    ('dropper', dropper(useless_feature)),
    ('encoder', custom_encoder())
])

In [3]:
dumy_pre = Pipeline([
    ('add_feature', add_new_feature()),
    ('dropper', dropper(useless_feature+['Race&T3', 'Family&Iodine', 'T3&T4'])), 
    ('encoder', custom_encoder())
])

# 3. 모델 학습 및 교차검증 (10-fold)

1. StratifiedKFold를 이용해 클래스 비율을 일정하게 유지하면서 교차검증을 시행
2. 각 fold를 검증 셋으로 사용하여 파생 특성 추가 전 / 후의 성능을 기록
3. 파생 특성 추가 전 / 후의 성능에 대해 대응표본 t-검정을 시행


In [4]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

accs_preprocessor = []
accs_dumy_pre = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

print("       특성 추가 후          특성 추가 전")
print("=============================================================")
print("Fold | Acc(pre)            | Acc(dumy)")
print("-------------------------------------------------------------")

for fold, (train_idx, val_idx) in enumerate(skf.split(x_train, y_train), start=1):
    x_tr, x_val = x_train.iloc[train_idx], x_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # ====== preprocessor ======
    x_tr_pre = preprocessor.fit_transform(x_tr)
    x_val_pre = preprocessor.transform(x_val)
    smote = SMOTE(random_state=42)
    x_smote, y_smote = smote.fit_resample(x_tr_pre, y_tr)

    model1 = RandomForestClassifier(random_state=42)
    model1.fit(x_smote, y_smote)
    y_pred_pre = model1.predict(x_val_pre)

    acc_pre = accuracy_score(y_val, y_pred_pre)
    accs_preprocessor.append(acc_pre)
   
    # ====== dumy_pre ======
    x_tr_dumy = dumy_pre.fit_transform(x_tr)
    x_val_dumy = dumy_pre.transform(x_val)
    smote_dumy = SMOTE(random_state=42)
    x_smote_dumy, y_smote_dumy = smote_dumy.fit_resample(x_tr_dumy, y_tr)

    model2 = RandomForestClassifier(random_state=42)
    model2.fit(x_smote_dumy, y_smote_dumy)
    y_pred_dumy = model2.predict(x_val_dumy)

    acc_dumy = accuracy_score(y_val, y_pred_dumy)
    accs_dumy_pre.append(acc_dumy)
    
    # 각 fold 결과 출력
    print(f"{fold:>4} |  {acc_pre:.4f}             |   {acc_dumy:.4f}")

       특성 추가 후          특성 추가 전
Fold | Acc(pre)            | Acc(dumy)
-------------------------------------------------------------
   1 |  0.8592             |   0.8369
   2 |  0.8546             |   0.8340
   3 |  0.8498             |   0.8329
   4 |  0.8636             |   0.8494
   5 |  0.8540             |   0.8290
   6 |  0.8586             |   0.8394
   7 |  0.8561             |   0.8315
   8 |  0.8606             |   0.8429
   9 |  0.8623             |   0.8393
  10 |  0.8544             |   0.8448


# 4. 결과 출력

In [5]:
print("=== 전체 평균 성능 비교 ===")
print("파생 특성 추가 전")
print(f"정확도 : {np.mean(accs_dumy_pre).round(3)} ±{np.std(accs_dumy_pre).round(3)}")

print()
print("파생 특성 추가 후")
print(f"정확도 : {np.mean(accs_preprocessor).round(3)} ±{np.std(accs_preprocessor).round(3)}")

=== 전체 평균 성능 비교 ===
파생 특성 추가 전
정확도 : 0.838 ±0.006

파생 특성 추가 후
정확도 : 0.857 ±0.004


# 5 대응 표본 t-검정 시행

귀무가설 : ___파생변수 추가 전후의 성능 변화가 없을 것이다___   
대립가설 : ___파생변수 추가 전후의 유의미한 성능 증가가 있을 것이다___   

In [6]:
from scipy.stats import ttest_rel
import numpy as np

# numpy 배열로 변환
acc_pre = np.array(accs_preprocessor)
acc_dumy = np.array(accs_dumy_pre)

# === Accuracy 단측 t-test ===
t_stat_acc, p_val_acc_two_sided = ttest_rel(acc_pre, acc_dumy)
p_val_acc_one_sided = p_val_acc_two_sided / 2
print("===== 단측 대응 표본 t-검정 결과 =====")
print(f"[Accuracy]  t-stat: {t_stat_acc:.4f} | p-value: {p_val_acc_one_sided:.4f}")
if p_val_acc_one_sided < 0.05 and t_stat_acc > 0:
    print("Accuracy는 preprocessor가 유의미하게 더 높습니다.")
else:
    print("Accuracy 차이는 유의하지 않습니다.")

===== 단측 대응 표본 t-검정 결과 =====
[Accuracy]  t-stat: 12.5299 | p-value: 0.0000
Accuracy는 preprocessor가 유의미하게 더 높습니다.


# 6. 하이퍼 파라미터 튜닝

In [7]:
# GridSearchCV를 통한 하이퍼파라미터 튜닝
param_grid = {
    'n_estimators': [100, 200], # 랜덤 포레스트의 사용할 트리의 개수
    'max_depth': [10, None], # 개별 트리의 최대 깊이
    'min_samples_split': [2, 5, 10], # 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4], # 리프 노드가 되기 위한 최소 샘플 
    'max_features': ['sqrt', 'log2'], # 각 노드를 분할할 때 고려할 특성의 개수
    'class_weight': ['balanced', None] # balanced: 클래스에 반비례하는 가중치 자동 부여 / None: 가중치 없음
}
# 파생변수 추가 된 전처리기로 전처리
x_train_processed = preprocessor.fit_transform(x_train)
x_test_processed = preprocessor.transform(x_test)

smote = SMOTE(random_state=42)
x_train_smote, y_train_smote = smote.fit_resample(x_train_processed, y_train)

rf_grid = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,  # 5-fold cross validation
    scoring="roc_auc",  # roc_auc로 평가
    n_jobs=-1,  # 모든 CPU 코어 사용
    verbose=1 # 각 조합에 대한 진행 상황 출력
)

rf_grid.fit(x_train_smote, y_train_smote)

# 최적 파라미터 출력
print()
print(f"Best parameters: {rf_grid.best_params_}")
best_model = rf_grid.best_estimator_
y_pred_optimized = best_model.predict(x_test_processed)

Fitting 5 folds for each of 144 candidates, totalling 720 fits

Best parameters: {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


# 7. 모델의 튜닝 전후 정확도 차이

In [8]:
# 튜닝한 모델에 대한 교차 검증
accs_best_model = []
for fold, (train_idx, val_idx) in enumerate(skf.split(x_train, y_train), start=1):
    x_tr, x_val = x_train.iloc[train_idx], x_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    x_tr_proc = preprocessor.fit_transform(x_tr)
    x_val_proc = preprocessor.transform(x_val)
    x_tr_proc, y_tr = SMOTE(random_state=42).fit_resample(x_tr_proc, y_tr)

    model_best = RandomForestClassifier(**rf_grid.best_params_, random_state=42)
    model_best.fit(x_tr_proc, y_tr)

    y_val_pred = model_best.predict(x_val_proc)
    acc_best = accuracy_score(y_val, y_val_pred)
    accs_best_model.append(acc_best)
    print(f"fold {fold} accuracy |  {acc_best:.4f}")

fold 1 accuracy |  0.8646
fold 2 accuracy |  0.8588
fold 3 accuracy |  0.8550
fold 4 accuracy |  0.8694
fold 5 accuracy |  0.8627
fold 6 accuracy |  0.8648
fold 7 accuracy |  0.8598
fold 8 accuracy |  0.8675
fold 9 accuracy |  0.8667
fold 10 accuracy |  0.8585


In [9]:
print(f"정확도 : {np.mean(accs_best_model).round(3)} ±{np.std(accs_best_model).round(3)}")

정확도 : 0.863 ±0.004


## 튜닝 전과 튜닝 후의 t-검정
귀무가설 : ___튜닝 전과 튜닝 후의 평균 차이는 없다___   
대립가설 : ___튜닝 후 모델이 정확도 평균이 높다___   

In [10]:
# 튜닝 전과 튜닝 후 사이의 통계 검정
t_stat, p_val = ttest_rel(accs_best_model, accs_preprocessor) # 튜닝 후: accs_best_model / 튜닝 전: accs_preprocessor

print("\n===== 튜닝 전후 정확도에 대한 t-검정 =====")
print(f"t-stat: {t_stat:.4f}, p-value: {p_val}")
if p_val < 0.05 and t_stat > 0:
    print("튜닝된 모델이 유의미하게 더 정확도가 높습니다.")
else:
    print("튜닝 후 정확도 향상은 통계적으로 유의하지 않습니다.")


===== 튜닝 전후 정확도에 대한 t-검정 =====
t-stat: 11.3173, p-value: 1.266128374775973e-06
튜닝된 모델이 유의미하게 더 정확도가 높습니다.


# 8. 베이스 모델과 최적화된 모델의 성능 차이

In [11]:
# 베이스 모델 최종 학습 - (파생변수 추가가 없는 모델)
x_train_dumy = dumy_pre.fit_transform(x_train)
x_test_dumy = dumy_pre.transform(x_test)

smote = SMOTE(random_state=42)
x_train_dumy_smote, y_train_dumy_smote = smote.fit_resample(x_train_dumy, y_train)
model2_final = RandomForestClassifier(random_state=42)
model2_final.fit(x_train_dumy_smote, y_train_dumy_smote)
y_pred_dumy_final = model2_final.predict(x_test_dumy)
acc_dumy_final = accuracy_score(y_test, y_pred_dumy_final)

# 튜닝 후 모델 최종 학습
x_train_processed = preprocessor.fit_transform(x_train)
x_test_processed = preprocessor.transform(x_test)
x_train_smote, y_train_smote = SMOTE(random_state=42).fit_resample(x_train_processed, y_train)
best_model_final = RandomForestClassifier(**rf_grid.best_params_, random_state=42)
best_model_final.fit(x_train_smote, y_train_smote)

# 예측 및 정확도 계산
y_pred_best_final = best_model_final.predict(x_test_processed)
acc_best_final = accuracy_score(y_test, y_pred_best_final)


# 비교 출력
# --------------------------------------------------
print("========= 최종 테스트셋 성능 비교 =========")
print(f"파생변수 없는 모델 (base model)     정확도 : {acc_dumy_final:.4f}")
print(f"파생변수 + 튜닝된 모델 (best model) 정확도 : {acc_best_final:.4f}")
print(f"정확도 차이 (best - base) : {(acc_best_final - acc_dumy_final):.4f}")

========= 최종 테스트셋 성능 비교 =========
파생변수 없는 모델 (base model)     정확도 : 0.8364
파생변수 + 튜닝된 모델 (best model) 정확도 : 0.8596
정확도 차이 (best - base) : 0.0233


# 9. 파생변수들에 대한 가설 검정

In [12]:
from scipy.stats import chi2_contingency
def t4_category(x):
    if x < 6:
        return 'T4_Low'
    elif x > 11.98:
        return 'T4_High'
    else:
        return 'T4_Normal'
def t3_category(x):
    if x < 1.4:
        return 'T3_Low'
    elif x > 3:
        return 'T3_High'
    else:
        return 'T3_Normal'
data = pd.read_csv("train.csv") # 전체 데이터셋에 대하여 카이제곱 검정
data["T3_Cat"] = data["T3_Result"].apply(t3_category)
data["T4_Cat"] = data["T4_Result"].apply(t4_category)
data["T3&T4"] = data["T3_Cat"] + "_" + data["T4_Cat"]
data['Race_T3'] = data['Race'].astype(str) + data['T3_Cat'].astype(str)
data['Race_T3'] = data['Race_T3'].astype('object')
data['Family&Iodine'] = data['Family_Background'].astype(str) + "_" + data['Iodine_Deficiency'].astype(str)

In [13]:
contingency_table = pd.crosstab(data["T3_Cat"], data["Cancer"])
chi2_stat, p, dof, expected = chi2_contingency(contingency_table)
print(f"T3호르몬 수치 -> 갑상선 암 발병")
print(f"카이제곱 통계량: {chi2_stat:.4f}")
print(f"p-value: {p: .4f}")

contingency_table = pd.crosstab(data["T4_Cat"], data["Cancer"])
chi2_stat, p, dof, expected = chi2_contingency(contingency_table)
print()
print(f"T4호르몬 수치 -> 갑상선 암 발병")
print(f"카이제곱 통계량: {chi2_stat:.4f}")
print(f"p-value: {p: .4f}")

contingency_table = pd.crosstab(data["T3&T4"], data["Cancer"])
chi2_stat, p, dof, expected = chi2_contingency(contingency_table)
print()
print(f"T3호르몬&T4호르몬 -> 갑상선 암 발병")
print(f"카이제곱 통계량: {chi2_stat:.4f}")
print(f"p-value: {p: .4f}")

T3호르몬 수치 -> 갑상선 암 발병
카이제곱 통계량: 1.3974
p-value:  0.4972

T4호르몬 수치 -> 갑상선 암 발병
카이제곱 통계량: 1.6389
p-value:  0.4407

T3호르몬&T4호르몬 -> 갑상선 암 발병
카이제곱 통계량: 19.1033
p-value:  0.0143


In [14]:
contingency_table = pd.crosstab(data['T3_Cat'], data['Cancer'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"T3호르몬 수치 -> 갑상선 암 발병")
print(f"카이제곱 통계량: {chi2:.4f}")
print(f"p-value: {p:.4f}")

contingency_table = pd.crosstab(data['Race'], data['Cancer'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print()
print(f"인종 -> 갑상선 암 발병")
print(f"카이제곱 통계량: {chi2:.4f}")
print(f"p-value: {p:.4f}")

contingency_table = pd.crosstab(data['Race_T3'], data['Cancer'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print()
print(f"T3호르몬 수치 & 인종 -> 갑상선 암 발병")
print(f"카이제곱 통계량: {chi2:.4f}")
print(f"p-value: {p:.4f}")

T3호르몬 수치 -> 갑상선 암 발병
카이제곱 통계량: 1.3974
p-value: 0.4972

인종 -> 갑상선 암 발병
카이제곱 통계량: 1384.2394
p-value: 0.0000

T3호르몬 수치 & 인종 -> 갑상선 암 발병
카이제곱 통계량: 1398.3145
p-value: 0.0000


In [15]:
contingency_table = pd.crosstab(data['Iodine_Deficiency'], data['Cancer'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"요오드 결핍 여부 -> 갑상선 암 발병")
print(f"카이제곱 통계량: {chi2:.4f}")
print(f"p-value: {p:.4f}")

contingency_table = pd.crosstab(data['Family_Background'], data['Cancer'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print()
print(f"가족력 여부 -> 갑상선 암 발병")
print(f"카이제곱 통계량: {chi2:.4f}")
print(f"p-value: {p:.4f}")

contingency_table = pd.crosstab(data['Family&Iodine'], data['Cancer'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print()
print(f"요오드 결핍 여부 & 가족력 여부 -> 갑상선 암 발병")
print(f"카이제곱 통계량: {chi2:.4f}")
print(f"p-value: {p:.4f}")

요오드 결핍 여부 -> 갑상선 암 발병
카이제곱 통계량: 556.3569
p-value: 0.0000

가족력 여부 -> 갑상선 암 발병
카이제곱 통계량: 1000.5772
p-value: 0.0000

요오드 결핍 여부 & 가족력 여부 -> 갑상선 암 발병
카이제곱 통계량: 1554.4941
p-value: 0.0000


# 10. 자료 및 코드 출처
1. DACON: [Baseline] SMOTE를 활용한 XGBoost 기반의 갑상선암 분류 中
2. ChatGPT: 통계 t-검정 관련 참조
3. Claude: 하이퍼 파라미터 튜닝 부분 中 GridSearchCV